### Домашнее задание

#### Аугментация изображений

Применить различные методы аугментации для работы с изображениями с использованием библиотеки `torchvision`. Цель — улучшить вариативность данных и повысить устойчивость будущих модели. Выбрать **5 аугментаций** из документации библиотеки `torchvision` - https://pytorch.org/vision/stable/transforms.html, применить их и сделайть выводы.

#### Примеры аугментаций

1. **Resize**: Изменение размера изображения до указанного размера.
2. **RandomShortestSize**: Преобразование размера изображения до случайной минимальной стороны, оставляя пропорции.
3. **RandomHorizontalFlip**: Случайное отражение изображения по горизонтали с вероятностью 50%.
4. **Pad**: Добавление паддинга (отступов) вокруг изображения.
5. **RandomAffine**: Случайное аффинное преобразование (масштабирование, повороты, смещения).
6. **RandomSolarize**: Солнечная коррекция изображения, которая меняет яркость пикселей выше определённого порога.
7. **ColorJitter**: Изменение яркости, контраста и насыщенности изображения.
8. **RandomRotation**: Случайное вращение изображения на угол от 0 до 30 градусов.
9. **RandomCrop**: Случайное обрезание изображения до меньшего размера.
10. **GaussianBlur**: Применение гауссового размытия.

#### Инструкция

1. **Выбор аугментаций.** Из списка выше или официальной документации библиотеки `torchvision` выбрать 5 различных аугментаций. Постараться выбрать различные типы аугментаций (например, изменение размера, поворот, цветовая коррекция и т.д.).
2. **Реализовать пайплайн аугментаций**
   - Использовать класс библиотеки `torchvision.transforms`, чтобы создать пайплайн аугментаций, который будет применяться к изображению.
   - Применять аугментации к изображению и отображать его перед и после изменений.
3. **Вывод.** Написать выводы.

### Решение

In [ ]:
pass